In [1]:
!pip install gower

import pandas as pd
import numpy as np
import pickle
import gower
from collections import namedtuple

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from sklearn.cluster import OPTICS, cluster_optics_dbscan

# OPTICS

We will run OPTICS with various values of min_samples, max_eps, and eps. Who only these three? After running models with various parameters, we found that configuring these three parameters has the most effect on model behaviors. Additionally, OPTICS takes time to run, so we want to cut down the number of parameter combinations to try. What about the other parameters?

- metric: this is the metric used for distance computation. Initially, we tested out various distance metrics that work both for OPTICS and ‘kd_tree’ (the algo we used to compute the nearest neighbors, explained later), and there was no significant impact on the clustering results. Therefore, it’s reasonable to use the default option, ‘minkowski.’ In a combination with the next parameter p=2, the ‘minkowski’ here is really the euclidean distance. However, since we want to make sure that euclidean distance is used, we explicitly specify the metric = ‘euclidean.’ For the model that uses the Gower distance, we specify the metric as 'precomputed.'
- p: this is the parameter for the Minkowski metric. The default =2 which sets the distance metric to euclidean_distance, which is what we used.
- metric_params: this is to add any additional keyword arguments for the metric function; since we used the default metric, we don’t have to add any more.
- cluster_method: this specifies the extraction method used to extract clusters using the calculated reachability and ordering. There are two options ‘xi’ and ‘dbscan.’ We tested out both options and found that ‘dbscan’ results in significantly better clusterings.
- xi: this determines the minimum steepness on the reachability plot that constitutes a cluster boundary. It is only used when the cluster method is ‘xi.’ Since we used dbscan, we don’t need to configure this param.
- predecessor_correction: This parameter has minimal effect on most datasets, and it is used only when cluster_method='xi,' therefore we did not use it.
- min_cluster_size: this param specifies the minimum number in an OPTICS cluster and it is only used when the cluster extraction method is ‘xi.’ Since we mainly used the ‘dbscan’ cluster extraction method, we did not use this param. It will be set as the same as the min_samples if it’s not specified.
- algorithm: This algorithm is used to compute the nearest neighbors. We choose ‘kd_tree’ as it works well for low-dimensional data.
- leaf_size: this param can affect the speed of the construction and query, as well as the memory required to store the tree. The default value is 30. Speed and memory were not our top priorities, therefore, we left it as default.
- memory: used to cache the output of the computation of the tree. By default, no caching is done. Similar to the previous param, not our top priority.

How did we choose the range of min_samples to try?
We ran models with min_samples from 5 to 220 with an increment of 5, then ran a final model with a min_samples of 250. We plotted the clustering with the tsne embeddings of the classifications from the previous project. We inspected these plots as the min_sample changed. We found that min_sample of 95 to 200 will result in more separated/clear clusters, with the ‘dbscan’ cluster extraction method. With the ‘xi’ extraction method, although as min_samples increased, the number of clusters decreased, when plotted against the classifications, the clusters were not clearly separated. We will run OPTICS with the Gower distance with min_samples in range (95, 201) with an increment of 15. We expect the clustering to be better than the OPTICS model fitted on the original unlabeled data.

How did we choose the range of max_eps to try?
Similar to the above, we ran models with max_eps from 0.2 to 1.5 with an increment of 0.2 and inspected the clustering plot. Max_eps is the maximum distance between two samples for one to be considered as in the neighborhood of the other. Initially, we did not specify this value. Without doing so, it will be set as np.inf (infinity), which will identify clusters across all scales. We found that with a max_eps of 1 or above, with the combination of other params, our models produce great results. We will run OPTICS with the Gower distance with max_eps in range (1, 2.1) with an increment of 0.5.

How did we choose the range of eps to try?
Eps indicates the maximum distance between two samples for one to be considered as in the neighborhood of the other. If non-specified, the default is set to be the same as the max_eps. This param is only used when the cluster extraction method is ‘dbscan,’ which is what we used. We run models with eps in a range of 0.2 to 1.5 with an increment of 0.2 and found that with eps greater than 1, the clusters produced were better. Therefore, we will use an eps in range (1, 2.1) with an increment of 0.5 to run OPTICS models with the Gower distance.

Load data:

In [3]:
y_LinearSVC = pd.read_pickle('./data/y_pred_LinearSVC.pkl')
y_LogReg = pd.read_pickle('./data/y_pred_LogReg.pkl')
y_RidgeReg = pd.read_pickle('./data/y_pred_RidgeReg.pkl')
X = pd.read_pickle('./data/unlabeled_behavior.pkl')
embedding = pd.read_pickle('./data/embedding.pkl')

Generate Gower matrix and data structure to store cluster assignments and scorings from parameter search:

In [4]:
X_gower = pd.DataFrame(gower.gower_matrix(X, cat_features=[False, False, False, False, True,
                                                           False, False, False, True]))

#define parameter ranges
#we originally wanted to search over the range commented below, but it took 15 hours of uninterrupted
#compute time to generate all of the models at min_samples=95, so we'll just run 95 and 200
min_samples_range = [200] #np.arange(95, 201, 15)
max_eps_range = np.arange(1.0, 2.1, 0.5)
eps_range = np.arange(1.0, 2.1, 0.5)

#create a data structure to store cluster assignments and scorings
Key = namedtuple('Key', ['s', 'me', 'e'])
'''
#This is how Labels_Scores was initialized, we had to interrupt execution and save it after the s=95
#runs completed.
Labels_Scores = {Key(s, me, e):{'labels_g':None, 'labels_e':None, 'relabels_g':None, 'relabels_e':None,
                                'score_e':{'lsvm':None, 'lr':None, 'rr':None},
                                'score_g':{'lsvm':None, 'lr':None, 'rr':None}}
                 for s, me, e in [[s, me, e] for s in min_samples_range
                                             for me in max_eps_range
                                             for e in eps_range]}
'''
#read in the partially filled Labels_Scores structure
with open('./data/labels_scores.pkl', 'rb') as handle:
    Labels_Scores = pickle.load(handle)

Define plotting function (this is modified from the version for TSNE):

In [5]:
# plot_with_ys(x1, x2, names, y_class, y_cluster, ext_names=False, relabeled=False, score=None)
# Generates a 2d plot of the given data. If only y_class is given, data points will be colored by
# classification. If y_cluster is also given, data points will be colored by cluster label with shapes
# corresponding to classification. Intended for use with embeddings generated by TSNE. Plots with
# extended names will go to a scratch folder so we can compare selections of parameters for OPTICS.
# Once we've chosen parameters these will be used through all future plots so the plots that go
# into our report can go without extended names.
# Variables:
# x1        -  array representing the position on the x-axis of each point in a 2d embedding from TSNE
# x2        -  array representing the position on the y-axis of each point in a 2d embedding from TSNE
# names     -  an array of string names for the classification algorithm used (index 0) and
#              the clustering algorithm used (index 1). If ext_names=True, also includes the 
#              min_samples (index 2), max_eps (index 3), eps (index 4), and the distance 
#              metric (index 5). For use in plotting.
# y_class   -  an array of classifications from a supervised model
# y_cluster -  an array of cluster labelings
# ext_names -  whether to look for extended names for the plot (default: False)
# relabeled -  whether clusters have been relabeled to enable computation of mutual information scores
#              (default: False)
# score     -  if relabeled=True, mutual information score of the clustering against the classification
#              (default: None)
@mpl.rc_context({'image.cmap': 'tab10', 'figure.figsize': [12.0, 8.0]})
def plot_with_ys(x1, x2, names, y_class, y_cluster, ext_names=False, relabeled=False, score=None):
    
    fig, ax = plt.subplots()
        
    #create a colormap of the correct size, doing this bc just giving 'tab10' to the cmap
    #parameter gives colors from each end of the palette instead of sequentially
    colors = mpl.colors.ListedColormap(plt.get_cmap('tab10')(np.arange(len(np.unique(y_cluster)))))

    #plot the two predicted classes with different markers, coloring by cluster assignment
    x1_normal = [a for a,b in zip(x1, y_class) if b == 0]
    x2_normal = [a for a,b in zip(x2, y_class) if b == 0]
    scatter1 = ax.scatter(x1_normal, x2_normal, marker='|', cmap=colors,
                          c=y_cluster[np.argwhere(y_class == 0)])

    x1_outlier = [a for a,b in zip(x1, y_class) if b == 1]
    x2_outlier = [a for a,b in zip(x2, y_class) if b == 1]
    scatter2 = ax.scatter(x1_outlier, x2_outlier, marker='_', cmap=colors,
                          c=y_cluster[np.argwhere(y_class == 1)])

    #create a legend for differentiating between colors
    legend1 = ax.legend(*scatter1.legend_elements(), loc="lower left", title="Clusters")
    ax.add_artist(legend1)

    #create a legend from scratch for differentiating between markers
    vline = mlines.Line2D([], [], color='black', marker='|', linestyle='None',
                          markersize=10, label='0 (normal)')
    hline = mlines.Line2D([], [], color='black', marker='_', linestyle='None',
                          markersize=10, label='1 (outlier)')
    legend2 = ax.legend(handles=[vline, hline], loc="lower right", title="Classes")

    #if relabeled: #add text reporting the mutual information score
        #TODO

    #insert given names to title and filename
    if ext_names: #with names for min_samples, max_eps, eps, and distance metric
        if relabeled: #plot has relabeled clusterings
            plt.title('TSNE Relabeled '+names[1]+' Clusterings & ' +names[0]+' Classifications s'
                      +names[2]+' me'+names[3]+' e'+names[4]+' '+names[5])
            filename = str('./figures/scratch/TSNE_relabeled_'+names[1]+'_'+names[0]+'_s'
                           +names[2]+'_me'+names[3]+'_e'+names[4]+'_'+names[5]+'.png')
        else: #plot has original clusterings
            plt.title('TSNE '+names[1]+' Clusterings & ' +names[0]+' Classifications s'
                      +names[2]+' me'+names[3]+' e'+names[4]+' '+names[5])
            filename = str('./figures/scratch/TSNE_'+names[1]+'_'+names[0]+'_s'+names[2]+'_me'
                           +names[3]+'_e'+names[4]+'_'+names[5]+'.png')

    else: #without names for min_samples, max_eps, eps, and distance metric
        if relabeled: #plot has relabeled clusterings
            plt.title('TSNE Relabeled '+names[1]+' Clusterings & ' +names[0]+' Classifications')
            filename = './figures/TSNE_relabeled_'+names[1]+'_'+names[0]+'.png'
        else:
            plt.title('TSNE '+names[1]+' Clusterings & ' +names[0]+' Classifications')
            filename = './figures/TSNE_'+names[1]+'_'+names[0]+'.png'

    plt.savefig(filename, format='png')

    plt.show()


Labels_Scores = 
Labels_Scores = Let's search over the range of parameters identified by our initial runs. We will generate and plot clusterings for each parameter combination, relabel clusterings so that the largest component is 0 (normal) and all others are 1 (outlier), compute mutual information scores against classifications, and replot with relabeled clusters and scores.

In [ ]:
%%capture

#first let's just compute the cluster assignments and plot them
for s in min_samples_range:
    
    for me in max_eps_range:
        
        #fit model with gower distance
        model_g = OPTICS(min_samples=s, max_eps=me, metric='precomputed',
                         algorithm='brute', n_jobs=-1)
        model_g.fit(X_gower)
        
        #fit model with euclidean distance
        model_e = OPTICS(min_samples=s, max_eps=me, metric='euclidean',
                         algorithm='kd_tree', n_jobs=-1)
        model_e.fit(X)
        
        for e in eps_range:
            
            #extract clusters 
            labels_g = cluster_optics_dbscan(reachability=model_g.reachability_,
                                             core_distances=model_g.core_distances_,
                                             ordering=model_g.ordering_, eps=e)
            labels_e = cluster_optics_dbscan(reachability=model_e.reachability_,
                                             core_distances=model_e.core_distances_,
                                             ordering=model_e.ordering_, eps=e)
            
            #add clusters to Labels_Scores dictionary
            Labels_Scores[Key(s, me, e)]['labels_g'] = labels_g
            Labels_Scores[Key(s, me, e)]['labels_e'] = labels_e
            
            #plot original clusterings for gower
            plot_with_ys(embedding[0], embedding[1], 
                         ['LinearSVC', 'OPTICS', str(s), str(me), str(e), 'gower'],
                         y_LinearSVC, labels_g, ext_names=True)
            plot_with_ys(embedding[0], embedding[1], 
                         ['LogReg', 'OPTICS', str(s), str(me), str(e), 'gower'],
                         y_LogReg, labels_g, ext_names=True)
            plot_with_ys(embedding[0], embedding[1], 
                         ['RidgeReg', 'OPTICS', str(s), str(me), str(e), 'gower'],
                         y_RidgeReg, labels_g, ext_names=True)
            
            #plot original clusterings for euclidean
            plot_with_ys(embedding[0], embedding[1], 
                         ['LinearSVC', 'OPTICS', str(s), str(me), str(e), 'euclidean'],
                         y_LinearSVC, labels_e, ext_names=True)
            plot_with_ys(embedding[0], embedding[1], 
                         ['LogReg', 'OPTICS', str(s), str(me), str(e), 'euclidean'],
                         y_LogReg, labels_e, ext_names=True)
            plot_with_ys(embedding[0], embedding[1], 
                         ['RidgeReg', 'OPTICS', str(s), str(me), str(e), 'euclidean'],
                         y_RidgeReg, labels_e, ext_names=True)

Pickle data structure of labels and scores:

In [ ]:
with open('./data/labels_scores.pkl', 'wb') as handle:
    pickle.dump(Labels_Scores, handle, protocol=pickle.HIGHEST_PROTOCOL)